In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_binary = pd.read_csv(r'.\TrafficLabelling\data_binary.csv')
data_multiclass = pd.read_csv(r'.\TrafficLabelling\data_multi_class.csv')

In [4]:
data_binary.head()

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [5]:
summary = data_binary.describe().T[['min','max','mean']]
# Hiển thị toàn bộ bảng (không bị cắt)
pd.set_option('display.max_rows', None)  # hiển thị tất cả dòng
pd.set_option('display.max_columns', None)  # hiển thị tất cả cột

# Hiển thị số ở dạng thường, làm tròn 2 chữ số thập phân
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print(summary)


                                            min              max          mean
 Destination Port                          0.00        65,535.00      8,071.48
 Protocol                                  0.00            17.00          9.88
 Flow Duration                           -13.00   119,999,998.00 14,785,663.93
 Total Fwd Packets                         1.00       219,759.00          9.36
 Total Backward Packets                    0.00       291,922.00         10.39
Total Length of Fwd Packets                0.00    12,900,000.00        549.30
 Total Length of Bwd Packets               0.00   655,453,030.00     16,162.64
 Fwd Packet Length Max                     0.00        24,820.00        207.60
 Fwd Packet Length Min                     0.00         2,325.00         18.71
 Fwd Packet Length Mean                    0.00         5,940.86         58.20
 Fwd Packet Length Std                     0.00         7,125.60         68.91
Bwd Packet Length Max                      0.00     

In [6]:
print("Tổng số giá trị thiếu:", data_binary.isnull().sum().sum())
print("Tổng số giá trị thiếu:", data_multiclass.isnull().sum().sum())

Tổng số giá trị thiếu: 0
Tổng số giá trị thiếu: 0


In [7]:
mask_negative = (data_binary.select_dtypes(include=['number']) < 0).any(axis=1)

# Lọc các hàng có ít nhất một giá trị âm
negative_rows = data_binary[mask_negative]

# # Hiển thị toàn bộ (không bị cắt)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# print(negative_rows)

# Nếu muốn, xuất ra CSV để xem rõ
negative_rows.to_csv("rows_with_negative_values.csv", index=False, encoding="utf-8-sig")

In [10]:
all_zero_cols = [col for col in data_binary.columns if (data_binary[col].nunique() == 1 and data_binary[col].iloc[0] == 0)]

print("Các cột toàn 0:")
for col in all_zero_cols:
    print(f"- {col}")


Các cột toàn 0:
-  Bwd PSH Flags
-  Bwd URG Flags
- Fwd Avg Bytes/Bulk
-  Fwd Avg Packets/Bulk
-  Fwd Avg Bulk Rate
-  Bwd Avg Bytes/Bulk
-  Bwd Avg Packets/Bulk
- Bwd Avg Bulk Rate


In [11]:
data_binary = data_binary.drop(columns=all_zero_cols)
data_multiclass = data_multiclass.drop(columns=all_zero_cols)

In [13]:
summary = data_binary.describe().T[['min','max','mean']]
summary.to_csv("summary_after_removing_zero_columns.csv", index=True, encoding="utf-8-sig")


In [15]:
cols_no_negative = [
    'Total Fwd Packets','Total Backward Packets',
    'Total Length of Fwd Packets','Total Length of Bwd Packets',
    'Fwd Packet Length Max','Fwd Packet Length Min','Fwd Packet Length Mean','Fwd Packet Length Std',
    'Bwd Packet Length Max','Bwd Packet Length Min','Bwd Packet Length Mean','Bwd Packet Length Std',
    'FIN Flag Count','SYN Flag Count','RST Flag Count','PSH Flag Count','ACK Flag Count',
    'URG Flag Count','CWE Flag Count','ECE Flag Count',
    'Subflow Fwd Packets','Subflow Fwd Bytes','Subflow Bwd Packets','Subflow Bwd Bytes',
    'act_data_pkt_fwd','Average Packet Size','Avg Fwd Segment Size','Avg Bwd Segment Size'
    # thêm các cột khác luôn ≥0
]
data_binary.columns = data_binary.columns.str.strip()
# 2️⃣ Tạo mask cho các row có giá trị âm ở các cột này
mask_invalid_negative = (data_binary[cols_no_negative] < 0).any(axis=1)

# 3️⃣ Lọc ra chỉ giữ row hợp lý

data_cleaned = data_binary[~mask_invalid_negative]
print("Shape của dataset sau khi làm sạch:", data_cleaned.shape)

# 2️⃣ Tổng số mẫu theo từng label
# Giả sử cột label tên là 'Label'
label_counts = data_cleaned['Label'].value_counts()
print("\nTổng số mẫu theo từng label:")
print(label_counts)


Shape của dataset sau khi làm sạch: (2830743, 72)

Tổng số mẫu theo từng label:
Label
BENIGN    2273097
ATTACK     557646
Name: count, dtype: int64
